In [ ]:
# Install required packages
!pip install ultralytics supervision

In [15]:
# Import libraries
import cv2
from ultralytics import YOLO
import supervision as sv
import time
import torch
from google.colab import drive
import os
from tqdm.notebook import tqdm

In [18]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# Set up project directories
PROJECT_DIR = '/content/drive/MyDrive/face_detection_project'
MODEL_DIR = f'{PROJECT_DIR}/models'
VIDEO_DIR = f'{PROJECT_DIR}/videos'
OUTPUT_DIR = f'{PROJECT_DIR}/outputs'

In [20]:
# Create directories if they don't exist
for dir_path in [MODEL_DIR, VIDEO_DIR, OUTPUT_DIR]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [21]:
# Check for GPU
print("Using GPU:", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"

Using GPU: True


In [22]:
def detect_people(video_name, model_name):
    """
    Process video with face detection model

    Args:
        video_name: Name of video file in VIDEO_DIR
        model_name: Name of model file in MODEL_DIR
    """
    video_path = os.path.join(VIDEO_DIR, video_name)
    model_path = os.path.join(MODEL_DIR, model_name)
    output_path = os.path.join(OUTPUT_DIR, f'processed_{video_name}')

    # Check if files exist
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video file not found: {video_path}")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")

    # Load YOLO model
    model = YOLO(model_path).to(device)

    # Initialize video capture
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create output video writer
    out = cv2.VideoWriter(output_path,
                         cv2.VideoWriter_fourcc(*'mp4v'),
                         fps,
                         (width, height))

    # Create BoxAnnotator instance
    box_annotator = sv.BoxAnnotator(
        color=sv.Color(r=200, g=50, b=200),
        thickness=3
    )

    # Process frames with progress bar
    for _ in tqdm(range(total_frames), desc="Processing video"):
        ret, frame = cap.read()
        if not ret:
            break

        # Resize frame for processing (but keep original for output)
        scale_percent = 40
        resized_width = int(width * scale_percent / 100)
        resized_height = int(height * scale_percent / 100)
        resized_frame = cv2.resize(frame, (resized_width, resized_height))

        # Detect people
        results = model(resized_frame, classes=[0])[0]

        # Convert detections to supervision format
        detections = sv.Detections.from_ultralytics(results)

        # Scale detections back to original size
        if len(detections) > 0:
            detections.xyxy = detections.xyxy * (width/resized_width)

        # Annotate the original frame
        annotated_frame = box_annotator.annotate(
            scene=frame,
            detections=detections
        )

        # Add detection count
        detection_count = len(detections)
        cv2.putText(
            annotated_frame,
            f"People detected: {detection_count}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0),
            2
        )

        # Write frame to output video
        out.write(annotated_frame)

    # Release resources
    cap.release()
    out.release()

    print(f"Processing complete! Output saved to: {output_path}")

## Manually copy model file (best.pt) to /content/drive/MyDrive/face_detection_project/models/

## Copy video files to /content/drive/MyDrive/face_detection_project/videos/

In [ ]:
if __name__ == "__main__":
    # First, manually copy your video and model files to the respective directories in Google Drive (see above)
    # Then run:
    detect_people("4.mp4", "best.pt")

## The processed videos will be saved in:  
/content/drive/MyDrive/face_detection_project/outputs/